# **Task 5: Mental Health Support Chatbot (Fine-Tuned)**

In [ ]:
pip install -q transformers datasets torch accelerate

In [ ]:
pip install readline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for readline: filename=readline-6.2.4.1-cp312-cp312-linux_x86_64.whl size=352246 sha256=868e0ca8a4f4ffbd88a6abc8950a448ffd9ad7e0e57fc549ff44eab60e181d66
  Stored in directory: /root/.cache/pip/wheels/22/57/93/854f8a4a4a9fc1e355388921a59e64f69000a34744cb154aa9
Successfully built readline


In [ ]:
pip install nltk

In [ ]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 48.6 MB/s eta 0:00:00


In [ ]:
!pip uninstall -y datasets
!pip install datasets==2.14.5

Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
from datasets import load_dataset

dataset = load_dataset("facebook/empathetic_dialogues")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/76673 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10943 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 76673
    })
    validation: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 12030
    })
    test: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 10943
    })
})


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def preprocess_for_empathy(batch):
    input_ids = []
    attention_masks = []
    labels = []

    for context, utterance in zip(batch["context"], batch["utterance"]):
        # Convert list of turns into a single string
        context_text = " ".join(context)

        # Gentle & empathetic prompt format
        prompt = f"Context: {context_text}\nResponse:"
        full_text = prompt + " " + utterance

        tokenized = tokenizer(
            full_text,
            truncation=True,
            padding="max_length",
            max_length=96
        )

        input_ids.append(tokenized["input_ids"])
        attention_masks.append(tokenized["attention_mask"])
        labels.append(tokenized["input_ids"].copy())  # causal LM

    return {
        "input_ids": input_ids,
        "attention_masks": attention_masks,
        "labels": labels
    }



In [ ]:
tokenized_dataset = dataset.map(
    preprocess_for_empathy,
    batched=True)

Map:   0%|          | 0/76673 [00:00<?, ? examples/s]

Map:   0%|          | 0/12030 [00:00<?, ? examples/s]

Map:   0%|          | 0/10943 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags', 'input_ids', 'attention_masks', 'labels'],
        num_rows: 76673
    })
    validation: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags', 'input_ids', 'attention_masks', 'labels'],
        num_rows: 12030
    })
    test: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags', 'input_ids', 'attention_masks', 'labels'],
        num_rows: 10943
    })
})


In [ ]:
import torch
from transformers import Trainer, TrainingArguments
import os
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir="./empathetic_chatbot",
    overwrite_output_dir=True,
    eval_strategy="steps", # Changed from evaluation_strategy
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    learning_rate=5e-5,
    fp16=False,
    logging_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(5000)),
    eval_dataset = tokenized_dataset["validation"].shuffle(seed=42).select(range(1000)),
    tokenizer=tokenizer
)


trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-3616614526.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
500,0.722600,0.727957
1000,0.736000,0.718761
1500,0.691400,0.712308
2000,0.677600,0.704183
2500,0.651900,0.700400
3000,0.572600,0.704920
3500,0.587900,0.701761
4000,0.622000,0.699254
4500,0.582300,0.699032
5000,0.576200,0.697396


TrainOutput(global_step=5000, training_loss=0.6681102256774902, metrics={'train_runtime': 463.6573, 'train_samples_per_second': 21.568, 'train_steps_per_second': 10.784, 'total_flos': 244965703680000.0, 'train_loss': 0.6681102256774902, 'epoch': 2.0})

In [ ]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.697396457195282, 'eval_runtime': 5.7177, 'eval_samples_per_second': 174.896, 'eval_steps_per_second': 87.448, 'epoch': 2.0}


In [ ]:
model.save_pretrained("./empathetic_chatbot/final_model")
tokenizer.save_pretrained("./empathetic_chatbot/final_model")


('./empathetic_chatbot/final_model/tokenizer_config.json',
 './empathetic_chatbot/final_model/special_tokens_map.json',
 './empathetic_chatbot/final_model/vocab.json',
 './empathetic_chatbot/final_model/merges.txt',
 './empathetic_chatbot/final_model/added_tokens.json',
 './empathetic_chatbot/final_model/tokenizer.json')

In [ ]:
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("./empathetic_chatbot/final_model")
model = AutoModelForCausalLM.from_pretrained("./empathetic_chatbot/final_model")

st.title("🧠💬 Mental Health Support Chatbot")

user_input = st.text_input("How are you feeling today?")

if user_input:
    prompt = f"Context: {user_input}\nResponse:"
    inputs = tokenizer(prompt, return_tensors="pt")

    output = model.generate(
        **inputs,
        max_length=120,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    st.write("**Bot:**", response)


2025-12-22 15:22:25.554 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-22 15:22:25.697 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-12-22 15:22:25.698 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-22 15:22:25.699 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-22 15:22:25.701 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-22 15:22:25.702 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-22 15:22:25.703 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-22 15:22:25.704 Thread 'MainThread': mi